In [1]:
!pip install pyomo -q
!pip install mistralai -q
!wget -N -q "https://matematica.unipv.it/gualandi/solvers/ipopt-linux64.zip"
!unzip -o -q ipopt-linux64

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 38.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.3 MB/s eta 0:00:00


In [2]:
import os
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage
from IPython.display import display, Markdown, Latex
from datetime import datetime

In [3]:
API_KEY = ''

MODEL_ID = 'open-mixtral-8x22b'
MODEL_SEED = 2
MODEL_TEMPERATURE = 0.7

SYSTEM_PROMPT_1 = """Please formulate a mathematical optimization model for this problem.
It is important that you do this by following these steps:

1. Define parameters and variables
2. Define the objective function
3. Define the constraints

Please note that each answer is always dedicated to one step. Therefore, do not provide extra information than what is asked. The answers will be manually combined by the user to create the full model."""
SYSTEM_PROMPT_2 = """Please write a python pyomo code for this optimization problem. Use sample data where needed. Indicate where you use sample data."""
USER_PROMPT_1 = """We are now at the first step."""
USER_PROMPT_2 = """We are now at the second step."""
USER_PROMPT_3 = """We are now at the third step."""
PROBLEM_DESCRIPTION = """We are looking at an alkylation process which will include the following 10 variables: olefin feed (barrels per day), isobutane recycle (barrels per day), acid addition rate (thousands of pounds per day), alkylate yield (barrels per day), isobutane makeup (barrels per day), acid strength (weight per cent), motor octane number, external isobutane-to-olefin ratio, acid dilution factor and F-4 performance number.

We want to maximize the daily profit of this alkylation process.
The profit is defined as the revenue generated from the alkylate yield multiplied with the motor octane number, minus the operational costs, which include olefin feed, isobutane recycle, acid addition rate, and isobutane makeup.

Relationships in terms of other variables for alkylate yield, motor octane number, acid dilution factor, and F-4 performance number can be formulated as regression formulas.
This regression estimate can deviate in both directions from true value of these variables by 2, 1, 5 and 10 percent, respectively.
Alkylate yield is a function of olefin feed and external isobutane-to-olefine yield. Alkalyte yield equals the amount of olefin feed multiplied by the sum of 1.12, 0.13167 times the external isobutane-to-olefin ratio and -0.00667 times the external isobutane-to-olefin ratio squared.
The motor octane number is derived from the external isobutane-to-olefin ratio and the acid strength. The motor octane number is calculated as the sum of 86.35, 1.098 time external isobutane-to-olefin ratio, -0.038 times the external isobutane-to-olefin ratio squared and 0.325 times acid strength reduced by 89.
The acid dilution factor is calculated based on the F-4 performance number. The acid dillution factor is expressed as 35.82 minus 0.222 times F-4 performance number.
Lastly, the F-4 performance number depends on the motor octane number. F-4 performance number is calculated as -133 plus three times the motor octane number.

There are some additional constraints imposed by the nature of the chemical process.
Each variable has a lower and an upper bound.
The external isobutane-to-olefin ratio needs to equal the ratio of isobutane recycle plus isobutane makeup to olefin feed.
The acid strength needs to equal the ratio of 93000 times acid addition rate to acid addition rate multiplied by acid dilution factor in addition to 1000 times acid addition rate.
Lastly, 1.22 alkylate yield needs to be equal to the combined olefin feed and isobutane makeup."""


client = MistralClient(api_key=API_KEY)

print(f'Time of execution: {datetime.now()}')

Time of execution: 2024-06-14 11:52:14.560052


## Step 1 - Generate Mathematical Formulation

In [4]:
messages_1 = [
    ChatMessage(role="system", content=SYSTEM_PROMPT_1),
    ChatMessage(role="user", content=PROBLEM_DESCRIPTION),
    ChatMessage(role="user", content=USER_PROMPT_1)
]

response_1 = client.chat(
    model=MODEL_ID,
    messages=messages_1,
    random_seed=MODEL_SEED,
    temperature=MODEL_TEMPERATURE
)

response_1_text = response_1.choices[0].message.content

In [5]:
Markdown(response_1_text)

Step 1: Define parameters and variables

Parameters:
- $c_{of}$: Cost per barrel of olefin feed
- $c_{ir}$: Cost per barrel of isobutane recycle
- $c_{aar}$: Cost per thousand pounds of acid addition rate
- $c_{im}$: Cost per barrel of isobutane makeup
- $p_{ay}$: Price per barrel of alkylate yield
- $p_{mon}$: Price per unit of motor octane number

Variables:
- $x_{of}$: Olefin feed (barrels per day)
- $x_{ir}$: Isobutane recycle (barrels per day)
- $x_{aar}$: Acid addition rate (thousand pounds per day)
- $x_{ay}$: Alkylate yield (barrels per day)
- $x_{im}$: Isobutane makeup (barrels per day)
- $x_{as}$: Acid strength (weight percent)
- $x_{mon}$: Motor octane number
- $x_{eir}$: External isobutane-to-olefin ratio
- $x_{adf}$: Acid dilution factor
- $x_{f4pn}$: F-4 performance number

Constants:
- $k_1 = 1.12$
- $k_2 = 0.13167$
- $k_3 = -0.00667$
- $k_4 = 86.35$
- $k_5 = 1.098$
- $k_6 = -0.038$
- $k_7 = 0.325$
- $k_8 = 35.82$
- $k_9 = -0.222$
- $k_{10} = -133$
- $k_{11} = 3$
- $k_{12} = 93000$
- $k_{13} = 1000$
- $k_{14} = 1.22$

In [6]:
print(response_1_text)

Step 1: Define parameters and variables

Parameters:
- $c_{of}$: Cost per barrel of olefin feed
- $c_{ir}$: Cost per barrel of isobutane recycle
- $c_{aar}$: Cost per thousand pounds of acid addition rate
- $c_{im}$: Cost per barrel of isobutane makeup
- $p_{ay}$: Price per barrel of alkylate yield
- $p_{mon}$: Price per unit of motor octane number

Variables:
- $x_{of}$: Olefin feed (barrels per day)
- $x_{ir}$: Isobutane recycle (barrels per day)
- $x_{aar}$: Acid addition rate (thousand pounds per day)
- $x_{ay}$: Alkylate yield (barrels per day)
- $x_{im}$: Isobutane makeup (barrels per day)
- $x_{as}$: Acid strength (weight percent)
- $x_{mon}$: Motor octane number
- $x_{eir}$: External isobutane-to-olefin ratio
- $x_{adf}$: Acid dilution factor
- $x_{f4pn}$: F-4 performance number

Constants:
- $k_1 = 1.12$
- $k_2 = 0.13167$
- $k_3 = -0.00667$
- $k_4 = 86.35$
- $k_5 = 1.098$
- $k_6 = -0.038$
- $k_7 = 0.325$
- $k_8 = 35.82$
- $k_9 = -0.222$
- $k_{10} = -133$
- $k_{11} = 3$
- $k_{1

In [7]:
messages_2 = [
    ChatMessage(role="system", content=SYSTEM_PROMPT_1),
    ChatMessage(role="user", content=PROBLEM_DESCRIPTION),
    ChatMessage(role="user", content=USER_PROMPT_1),
    ChatMessage(role="assistant", content=response_1_text),
    ChatMessage(role="user", content=USER_PROMPT_2),
]

response_2 = client.chat(
    model=MODEL_ID,
    messages=messages_2,
    random_seed=MODEL_SEED,
    temperature=MODEL_TEMPERATURE
)

response_2_text = response_2.choices[0].message.content

In [8]:
Markdown(response_2_text)

Step 2: Define the objective function

The objective function is to maximize the daily profit, which is calculated as the revenue generated from the alkylate yield multiplied with the motor octane number, minus the operational costs, which include olefin feed, isobutane recycle, acid addition rate, and isobutane makeup.

The revenue is calculated as:
- $R_{ay} = p_{ay} \cdot x_{ay}$
- $R_{mon} = p_{mon} \cdot x_{mon}$

The total revenue is:
- $R = R_{ay} + R_{mon}$

The operational costs are calculated as:
- $C_{of} = c_{of} \cdot x_{of}$
- $C_{ir} = c_{ir} \cdot x_{ir}$
- $C_{aar} = c_{aar} \cdot x_{aar}$
- $C_{im} = c_{im} \cdot x_{im}$

The total operational cost is:
- $C = C_{of} + C_{ir} + C_{aar} + C_{im}$

The daily profit is:
- $P = R - C$

The objective function is to maximize the daily profit:
- $\max(P)$

Please note that the regression estimate can deviate in both directions from the true value of these variables by 2%, 1%, 5%, and 10% for alkylate yield, motor octane number, acid dilution factor, and F-4 performance number, respectively. This information should be considered when formulating the objective function and constraints.

Next, we will define the constraints for this optimization problem.

Please proceed to the next step.

In [9]:
print(response_2_text)

Step 2: Define the objective function

The objective function is to maximize the daily profit, which is calculated as the revenue generated from the alkylate yield multiplied with the motor octane number, minus the operational costs, which include olefin feed, isobutane recycle, acid addition rate, and isobutane makeup.

The revenue is calculated as:
- $R_{ay} = p_{ay} \cdot x_{ay}$
- $R_{mon} = p_{mon} \cdot x_{mon}$

The total revenue is:
- $R = R_{ay} + R_{mon}$

The operational costs are calculated as:
- $C_{of} = c_{of} \cdot x_{of}$
- $C_{ir} = c_{ir} \cdot x_{ir}$
- $C_{aar} = c_{aar} \cdot x_{aar}$
- $C_{im} = c_{im} \cdot x_{im}$

The total operational cost is:
- $C = C_{of} + C_{ir} + C_{aar} + C_{im}$

The daily profit is:
- $P = R - C$

The objective function is to maximize the daily profit:
- $\max(P)$

Please note that the regression estimate can deviate in both directions from the true value of these variables by 2%, 1%, 5%, and 10% for alkylate yield, motor octane numbe

In [10]:
messages_3 = [
    ChatMessage(role="system", content=SYSTEM_PROMPT_1),
    ChatMessage(role="user", content=PROBLEM_DESCRIPTION),
    ChatMessage(role="user", content=USER_PROMPT_1),
    ChatMessage(role="assistant", content=response_1_text),
    ChatMessage(role="user", content=USER_PROMPT_2),
    ChatMessage(role="assistant", content=response_2_text),
    ChatMessage(role="user", content=USER_PROMPT_3),
]

response_3 = client.chat(
    model=MODEL_ID,
    messages=messages_3,
    random_seed=MODEL_SEED,
    temperature=MODEL_TEMPERATURE
)

response_3_text = response_3.choices[0].message.content

In [11]:
Markdown(response_3_text)

Step 3: Define the constraints

Constraints are imposed by the nature of the chemical process and are as follows:

1. The external isobutane-to-olefin ratio needs to equal the ratio of isobutane recycle plus isobutane makeup to olefin feed:
- $x_{eir} = \frac{x_{ir} + x_{im}}{x_{of}}$
1. The acid strength needs to equal the ratio of 93000 times acid addition rate to acid addition rate multiplied by acid dilution factor in addition to 1000 times acid addition rate:
- $x_{as} = \frac{k_{12} \cdot x_{aar}}{x_{aar} \cdot x_{adf} + k_{13} \cdot x_{aar}}$
1. The alkylate yield is a function of olefin feed and external isobutane-to-olefin ratio:
- $x_{ay} = x_{of} \cdot (k_1 + k_2 \cdot x_{eir} + k_3 \cdot x_{eir}^2)$
- Since the regression estimate can deviate in both directions from the true value of alkylate yield by 2%, we can introduce a parameter $\delta_{ay}$ to represent this deviation, such that $x_{ay} = x_{of} \cdot (k_1 + k_2 \cdot x_{eir} + k_3 \cdot x_{eir}^2) \cdot (1 \pm \delta_{ay})$, where $\delta_{ay} \in [0, 0.02]$.
1. The motor octane number is derived from the external isobutane-to-olefin ratio and the acid strength:
- $x_{mon} = k_4 + k_5 \cdot x_{eir} + k_6 \cdot x_{eir}^2 + k_7 \cdot (x_{as} - 89)$
- Since the regression estimate can deviate in both directions from the true value of motor octane number by 1%, we can introduce a parameter $\delta_{mon}$ to represent this deviation, such that $x_{mon} = k_4 + k_5 \cdot x_{eir} + k_6 \cdot x_{eir}^2 + k_7 \cdot (x_{as} - 89) \cdot (1 \pm \delta_{mon})$, where $\delta_{mon} \in [0, 0.01]$.
1. The acid dilution factor is calculated based on the F-4 performance number:
- $x_{adf} = k_8 - k_9 \cdot x_{f4pn}$
- Since the regression estimate can deviate in both directions from the true value of acid dilution factor by 5%, we can introduce a parameter $\delta_{adf}$ to represent this deviation, such that $x_{adf} = k_8 - k_9 \cdot x_{f4pn} \cdot (1 \pm \delta_{adf})$, where $\delta_{adf} \in [0, 0.05]$.
1. The F-4 performance number depends on the motor octane number:
- $x_{f4pn} = k_{10} + k_{11} \cdot x_{mon}$
- Since the regression estimate can deviate in both directions from the true value of F-4 performance number by 10%, we can introduce a parameter $\delta_{f4pn}$ to represent this deviation, such that $x_{f4pn} = k_{10} + k_{11} \cdot x_{mon} \cdot (1 \pm \delta_{f4pn})$, where $\delta_{f4pn} \in [0, 0.1]$.
1. The acid strength needs to be within the range [78, 99] weight percent:
- $78 \leq x_{as} \leq 99$
1. Each variable has a lower and an upper bound:
- $l_{of} \leq x_{of} \leq u_{of}$
- $l_{ir} \leq x_{ir} \leq u_{ir}$
- $l_{aar} \leq x_{aar} \leq u_{aar}$
- $l_{im} \leq x_{im} \leq u_{im}$
- $l_{as} \leq x_{as} \leq u_{as}$
- $l_{mon} \leq x_{mon} \leq u_{mon}$
- $l_{eir} \leq x_{eir} \leq u_{eir}$
- $l_{adf} \leq x_{adf} \leq u_{adf}$
- $l_{f4pn} \leq x_{f4pn} \leq u_{f4pn}$
1. Lastly, 1.22 alkylate yield needs to be equal to the combined olefin feed and isobutane makeup:
- $x_{ay} = 1.22 \cdot (x_{of} + x_{im})$

This completes the definition of the mathematical optimization model for this problem. The objective function is defined in Step 2, and the constraints are defined in Step 3. The next step would be to solve this optimization problem using an appropriate optimization solver.

In [12]:
print(response_3_text)

Step 3: Define the constraints

Constraints are imposed by the nature of the chemical process and are as follows:

1. The external isobutane-to-olefin ratio needs to equal the ratio of isobutane recycle plus isobutane makeup to olefin feed:
- $x_{eir} = \frac{x_{ir} + x_{im}}{x_{of}}$
1. The acid strength needs to equal the ratio of 93000 times acid addition rate to acid addition rate multiplied by acid dilution factor in addition to 1000 times acid addition rate:
- $x_{as} = \frac{k_{12} \cdot x_{aar}}{x_{aar} \cdot x_{adf} + k_{13} \cdot x_{aar}}$
1. The alkylate yield is a function of olefin feed and external isobutane-to-olefin ratio:
- $x_{ay} = x_{of} \cdot (k_1 + k_2 \cdot x_{eir} + k_3 \cdot x_{eir}^2)$
- Since the regression estimate can deviate in both directions from the true value of alkylate yield by 2%, we can introduce a parameter $\delta_{ay}$ to represent this deviation, such that $x_{ay} = x_{of} \cdot (k_1 + k_2 \cdot x_{eir} + k_3 \cdot x_{eir}^2) \cdot (1 \pm \delt

In [13]:
response_text = '\n'.join([response_1_text, response_2_text, response_3_text])

Markdown(response_text)

Step 1: Define parameters and variables

Parameters:
- $c_{of}$: Cost per barrel of olefin feed
- $c_{ir}$: Cost per barrel of isobutane recycle
- $c_{aar}$: Cost per thousand pounds of acid addition rate
- $c_{im}$: Cost per barrel of isobutane makeup
- $p_{ay}$: Price per barrel of alkylate yield
- $p_{mon}$: Price per unit of motor octane number

Variables:
- $x_{of}$: Olefin feed (barrels per day)
- $x_{ir}$: Isobutane recycle (barrels per day)
- $x_{aar}$: Acid addition rate (thousand pounds per day)
- $x_{ay}$: Alkylate yield (barrels per day)
- $x_{im}$: Isobutane makeup (barrels per day)
- $x_{as}$: Acid strength (weight percent)
- $x_{mon}$: Motor octane number
- $x_{eir}$: External isobutane-to-olefin ratio
- $x_{adf}$: Acid dilution factor
- $x_{f4pn}$: F-4 performance number

Constants:
- $k_1 = 1.12$
- $k_2 = 0.13167$
- $k_3 = -0.00667$
- $k_4 = 86.35$
- $k_5 = 1.098$
- $k_6 = -0.038$
- $k_7 = 0.325$
- $k_8 = 35.82$
- $k_9 = -0.222$
- $k_{10} = -133$
- $k_{11} = 3$
- $k_{12} = 93000$
- $k_{13} = 1000$
- $k_{14} = 1.22$
Step 2: Define the objective function

The objective function is to maximize the daily profit, which is calculated as the revenue generated from the alkylate yield multiplied with the motor octane number, minus the operational costs, which include olefin feed, isobutane recycle, acid addition rate, and isobutane makeup.

The revenue is calculated as:
- $R_{ay} = p_{ay} \cdot x_{ay}$
- $R_{mon} = p_{mon} \cdot x_{mon}$

The total revenue is:
- $R = R_{ay} + R_{mon}$

The operational costs are calculated as:
- $C_{of} = c_{of} \cdot x_{of}$
- $C_{ir} = c_{ir} \cdot x_{ir}$
- $C_{aar} = c_{aar} \cdot x_{aar}$
- $C_{im} = c_{im} \cdot x_{im}$

The total operational cost is:
- $C = C_{of} + C_{ir} + C_{aar} + C_{im}$

The daily profit is:
- $P = R - C$

The objective function is to maximize the daily profit:
- $\max(P)$

Please note that the regression estimate can deviate in both directions from the true value of these variables by 2%, 1%, 5%, and 10% for alkylate yield, motor octane number, acid dilution factor, and F-4 performance number, respectively. This information should be considered when formulating the objective function and constraints.

Next, we will define the constraints for this optimization problem.

Please proceed to the next step.
Step 3: Define the constraints

Constraints are imposed by the nature of the chemical process and are as follows:

1. The external isobutane-to-olefin ratio needs to equal the ratio of isobutane recycle plus isobutane makeup to olefin feed:
- $x_{eir} = \frac{x_{ir} + x_{im}}{x_{of}}$
1. The acid strength needs to equal the ratio of 93000 times acid addition rate to acid addition rate multiplied by acid dilution factor in addition to 1000 times acid addition rate:
- $x_{as} = \frac{k_{12} \cdot x_{aar}}{x_{aar} \cdot x_{adf} + k_{13} \cdot x_{aar}}$
1. The alkylate yield is a function of olefin feed and external isobutane-to-olefin ratio:
- $x_{ay} = x_{of} \cdot (k_1 + k_2 \cdot x_{eir} + k_3 \cdot x_{eir}^2)$
- Since the regression estimate can deviate in both directions from the true value of alkylate yield by 2%, we can introduce a parameter $\delta_{ay}$ to represent this deviation, such that $x_{ay} = x_{of} \cdot (k_1 + k_2 \cdot x_{eir} + k_3 \cdot x_{eir}^2) \cdot (1 \pm \delta_{ay})$, where $\delta_{ay} \in [0, 0.02]$.
1. The motor octane number is derived from the external isobutane-to-olefin ratio and the acid strength:
- $x_{mon} = k_4 + k_5 \cdot x_{eir} + k_6 \cdot x_{eir}^2 + k_7 \cdot (x_{as} - 89)$
- Since the regression estimate can deviate in both directions from the true value of motor octane number by 1%, we can introduce a parameter $\delta_{mon}$ to represent this deviation, such that $x_{mon} = k_4 + k_5 \cdot x_{eir} + k_6 \cdot x_{eir}^2 + k_7 \cdot (x_{as} - 89) \cdot (1 \pm \delta_{mon})$, where $\delta_{mon} \in [0, 0.01]$.
1. The acid dilution factor is calculated based on the F-4 performance number:
- $x_{adf} = k_8 - k_9 \cdot x_{f4pn}$
- Since the regression estimate can deviate in both directions from the true value of acid dilution factor by 5%, we can introduce a parameter $\delta_{adf}$ to represent this deviation, such that $x_{adf} = k_8 - k_9 \cdot x_{f4pn} \cdot (1 \pm \delta_{adf})$, where $\delta_{adf} \in [0, 0.05]$.
1. The F-4 performance number depends on the motor octane number:
- $x_{f4pn} = k_{10} + k_{11} \cdot x_{mon}$
- Since the regression estimate can deviate in both directions from the true value of F-4 performance number by 10%, we can introduce a parameter $\delta_{f4pn}$ to represent this deviation, such that $x_{f4pn} = k_{10} + k_{11} \cdot x_{mon} \cdot (1 \pm \delta_{f4pn})$, where $\delta_{f4pn} \in [0, 0.1]$.
1. The acid strength needs to be within the range [78, 99] weight percent:
- $78 \leq x_{as} \leq 99$
1. Each variable has a lower and an upper bound:
- $l_{of} \leq x_{of} \leq u_{of}$
- $l_{ir} \leq x_{ir} \leq u_{ir}$
- $l_{aar} \leq x_{aar} \leq u_{aar}$
- $l_{im} \leq x_{im} \leq u_{im}$
- $l_{as} \leq x_{as} \leq u_{as}$
- $l_{mon} \leq x_{mon} \leq u_{mon}$
- $l_{eir} \leq x_{eir} \leq u_{eir}$
- $l_{adf} \leq x_{adf} \leq u_{adf}$
- $l_{f4pn} \leq x_{f4pn} \leq u_{f4pn}$
1. Lastly, 1.22 alkylate yield needs to be equal to the combined olefin feed and isobutane makeup:
- $x_{ay} = 1.22 \cdot (x_{of} + x_{im})$

This completes the definition of the mathematical optimization model for this problem. The objective function is defined in Step 2, and the constraints are defined in Step 3. The next step would be to solve this optimization problem using an appropriate optimization solver.

In [14]:
print(response_text)

Step 1: Define parameters and variables

Parameters:
- $c_{of}$: Cost per barrel of olefin feed
- $c_{ir}$: Cost per barrel of isobutane recycle
- $c_{aar}$: Cost per thousand pounds of acid addition rate
- $c_{im}$: Cost per barrel of isobutane makeup
- $p_{ay}$: Price per barrel of alkylate yield
- $p_{mon}$: Price per unit of motor octane number

Variables:
- $x_{of}$: Olefin feed (barrels per day)
- $x_{ir}$: Isobutane recycle (barrels per day)
- $x_{aar}$: Acid addition rate (thousand pounds per day)
- $x_{ay}$: Alkylate yield (barrels per day)
- $x_{im}$: Isobutane makeup (barrels per day)
- $x_{as}$: Acid strength (weight percent)
- $x_{mon}$: Motor octane number
- $x_{eir}$: External isobutane-to-olefin ratio
- $x_{adf}$: Acid dilution factor
- $x_{f4pn}$: F-4 performance number

Constants:
- $k_1 = 1.12$
- $k_2 = 0.13167$
- $k_3 = -0.00667$
- $k_4 = 86.35$
- $k_5 = 1.098$
- $k_6 = -0.038$
- $k_7 = 0.325$
- $k_8 = 35.82$
- $k_9 = -0.222$
- $k_{10} = -133$
- $k_{11} = 3$
- $k_{1

## Step 2 - Generate the Pyomo Code

In [15]:
messages_4 = [
    ChatMessage(role="system", content=SYSTEM_PROMPT_2),
    ChatMessage(role="user", content=response_text)
]

In [16]:
response_4 = client.chat(
    model=MODEL_ID,
    messages=messages_4,
    random_seed=MODEL_SEED,
    temperature=MODEL_TEMPERATURE
)

response_4_text = response_4.choices[0].message.content

In [17]:
Markdown(response_4_text)

Here is a Python Pyomo code for the optimization problem:

```python
import pyomo.environ as pyo

# Define the model
model = pyo.ConcreteModel()

# Step 1: Define parameters and variables

# Define parameters (sample data)
model.c_of = 100  # Cost per barrel of olefin feed
model.c_ir = 50  # Cost per barrel of isobutane recycle
model.c_aar = 10  # Cost per thousand pounds of acid addition rate
model.c_im = 150  # Cost per barrel of isobutane makeup
model.p_ay = 1500  # Price per barrel of alkylate yield
model.p_mon = 100  # Price per unit of motor octane number

# Define constants
model.k1 = 1.12
model.k2 = 0.13167
model.k3 = -0.00667
model.k4 = 86.35
model.k5 = 1.098
model.k6 = -0.038
model.k7 = 0.325
model.k8 = 35.82
model.k9 = -0.222
model.k10 = -133
model.k11 = 3
model.k12 = 93000
model.k13 = 1000
model.k14 = 1.22

# Define variables
model.x_of = pyo.Var(within=pyo.NonNegativeReals, bounds=(100, 1000))  # Olefin feed
model.x_ir = pyo.Var(within=pyo.NonNegativeReals, bounds=(100, 1000))  # Isobutane recycle
model.x_aar = pyo.Var(within=pyo.NonNegativeReals, bounds=(10, 100))  # Acid addition rate
model.x_ay = pyo.Var(within=pyo.NonNegativeReals, bounds=(100, 1000))  # Alkylate yield
model.x_im = pyo.Var(within=pyo.NonNegativeReals, bounds=(10, 100))  # Isobutane makeup
model.x_as = pyo.Var(within=pyo.NonNegativeReals, bounds=(78, 99))  # Acid strength
model.x_mon = pyo.Var(within=pyo.NonNegativeReals, bounds=(85, 95))  # Motor octane number
model.x_eir = pyo.Var(within=pyo.NonNegativeReals)  # External isobutane-to-olefin ratio
model.x_adf = pyo.Var(within=pyo.NonNegativeReals)  # Acid dilution factor
model.x_f4pn = pyo.Var(within=pyo.NonNegativeReals)  # F-4 performance number

# Step 2: Define the objective function

# Define the revenue
model.R_ay = pyo.Expression(expr=model.p_ay * model.x_ay)  # Revenue from alkylate yield
model.R_mon = pyo.Expression(expr=model.p_mon * model.x_mon)  # Revenue from motor octane number
model.R = pyo.Expression(expr=model.R_ay + model.R_mon)  # Total revenue

# Define the operational costs
model.C_of = pyo.Expression(expr=model.c_of * model.x_of)  # Cost of olefin feed
model.C_ir = pyo.Expression(expr=model.c_ir * model.x_ir)  # Cost of isobutane recycle
model.C_aar = pyo.Expression(expr=model.c_aar * model.x_aar)  # Cost of acid addition rate
model.C_im = pyo.Expression(expr=model.c_im * model.x_im)  # Cost of isobutane makeup
model.C = pyo.Expression(expr=model.C_of + model.C_ir + model.C_aar + model.C_im)  # Total operational cost

# Define the daily profit
model.P = pyo.Expression(expr=model.R - model.C)  # Daily profit

# Define the objective function
model.obj = pyo.Objective(expr=model.P, sense=pyo.maximize)

# Step 3: Define the constraints

# Constraint 1
model.c1 = pyo.Constraint(expr=model.x_eir == (model.x_ir + model.x_im) / model.x_of)

# Constraint 2
model.c2 = pyo.Constraint(expr=model.x_as == (model.k12 * model.x_aar) / (model.x_aar * model.x_adf + model.k13 * model.x_aar))

# Constraint 3
model.c3 = pyo.Constraint(expr=model.x_ay == model.x_of * (model.k1 + model.k2 * model.x_eir + model.k3 * model.x_eir**2))

# Constraint 4
model.c4 = pyo.Constraint(expr=model.x_mon == model.k4 + model.k5 * model.x_eir + model.k6 * model.x_eir**2 + model.k7 * (model.x_as - 89))

# Constraint 5
model.c5 = pyo.Constraint(expr=model.x_adf == model.k8 - model.k9 * model.x_f4pn)

# Constraint 6
model.c6 = pyo.Constraint(expr=model.x_f4pn == model.k10 + model.k11 * model.x_mon)

# Constraint 7
model.c7 = pyo.Constraint(expr=78 <= model.x_as <= 99)

# Constraint 8
model.c8 = pyo.ConstraintList(
    [model.x_of >= 100, model.x_of <= 1000,
     model.x_ir >= 100, model.x_ir <= 1000,
     model.x_aar >= 10, model.x_aar <= 100,
     model.x_im >= 10, model.x_im <= 100,
     model.x_as >= 78, model.x_as <= 99,
     model.x_mon >= 85, model.x_mon <= 95,
     model.x_eir >= 0,
     model.x_adf >= 0,
     model.x_f4pn >= 0]
)

# Constraint 9
model.c9 = pyo.Constraint(expr=model.x_ay == 1.22 * (model.x_of + model.x_im))

# Solve the model
solver = pyo.SolverFactory('glpk')
results = solver.solve(model)

# Print the results
print(f"Daily profit: {model.obj():.2f}")
print("Variable values:")
for v in model.component_objects(pyo.Var, active=True):
    print(f"  {v.name}: {getattr(model, v.name)()}")
```

This code defines the optimization model, sets the objective function, and applies the specified constraints. The code then solves the model using the GLPK solver. Note that the `glpk` solver is used in this example, but you can use any other supported Pyomo solver. The results are then printed to the console.

In [18]:
print(response_4_text)

Here is a Python Pyomo code for the optimization problem:

```python
import pyomo.environ as pyo

# Define the model
model = pyo.ConcreteModel()

# Step 1: Define parameters and variables

# Define parameters (sample data)
model.c_of = 100  # Cost per barrel of olefin feed
model.c_ir = 50  # Cost per barrel of isobutane recycle
model.c_aar = 10  # Cost per thousand pounds of acid addition rate
model.c_im = 150  # Cost per barrel of isobutane makeup
model.p_ay = 1500  # Price per barrel of alkylate yield
model.p_mon = 100  # Price per unit of motor octane number

# Define constants
model.k1 = 1.12
model.k2 = 0.13167
model.k3 = -0.00667
model.k4 = 86.35
model.k5 = 1.098
model.k6 = -0.038
model.k7 = 0.325
model.k8 = 35.82
model.k9 = -0.222
model.k10 = -133
model.k11 = 3
model.k12 = 93000
model.k13 = 1000
model.k14 = 1.22

# Define variables
model.x_of = pyo.Var(within=pyo.NonNegativeReals, bounds=(100, 1000))  # Olefin feed
model.x_ir = pyo.Var(within=pyo.NonNegativeReals, bounds=(100, 10

### Code Executability

In [19]:
import pyomo.environ as pyo

# Define the model
model = pyo.ConcreteModel()

# Step 1: Define parameters and variables

# Define parameters (sample data)
model.c_of = 100  # Cost per barrel of olefin feed
model.c_ir = 50  # Cost per barrel of isobutane recycle
model.c_aar = 10  # Cost per thousand pounds of acid addition rate
model.c_im = 150  # Cost per barrel of isobutane makeup
model.p_ay = 1500  # Price per barrel of alkylate yield
model.p_mon = 100  # Price per unit of motor octane number

# Define constants
model.k1 = 1.12
model.k2 = 0.13167
model.k3 = -0.00667
model.k4 = 86.35
model.k5 = 1.098
model.k6 = -0.038
model.k7 = 0.325
model.k8 = 35.82
model.k9 = -0.222
model.k10 = -133
model.k11 = 3
model.k12 = 93000
model.k13 = 1000
model.k14 = 1.22

# Define variables
model.x_of = pyo.Var(within=pyo.NonNegativeReals, bounds=(100, 1000))  # Olefin feed
model.x_ir = pyo.Var(within=pyo.NonNegativeReals, bounds=(100, 1000))  # Isobutane recycle
model.x_aar = pyo.Var(within=pyo.NonNegativeReals, bounds=(10, 100))  # Acid addition rate
model.x_ay = pyo.Var(within=pyo.NonNegativeReals, bounds=(100, 1000))  # Alkylate yield
model.x_im = pyo.Var(within=pyo.NonNegativeReals, bounds=(10, 100))  # Isobutane makeup
model.x_as = pyo.Var(within=pyo.NonNegativeReals, bounds=(78, 99))  # Acid strength
model.x_mon = pyo.Var(within=pyo.NonNegativeReals, bounds=(85, 95))  # Motor octane number
model.x_eir = pyo.Var(within=pyo.NonNegativeReals)  # External isobutane-to-olefin ratio
model.x_adf = pyo.Var(within=pyo.NonNegativeReals)  # Acid dilution factor
model.x_f4pn = pyo.Var(within=pyo.NonNegativeReals)  # F-4 performance number

# Step 2: Define the objective function

# Define the revenue
model.R_ay = pyo.Expression(expr=model.p_ay * model.x_ay)  # Revenue from alkylate yield
model.R_mon = pyo.Expression(expr=model.p_mon * model.x_mon)  # Revenue from motor octane number
model.R = pyo.Expression(expr=model.R_ay + model.R_mon)  # Total revenue

# Define the operational costs
model.C_of = pyo.Expression(expr=model.c_of * model.x_of)  # Cost of olefin feed
model.C_ir = pyo.Expression(expr=model.c_ir * model.x_ir)  # Cost of isobutane recycle
model.C_aar = pyo.Expression(expr=model.c_aar * model.x_aar)  # Cost of acid addition rate
model.C_im = pyo.Expression(expr=model.c_im * model.x_im)  # Cost of isobutane makeup
model.C = pyo.Expression(expr=model.C_of + model.C_ir + model.C_aar + model.C_im)  # Total operational cost

# Define the daily profit
model.P = pyo.Expression(expr=model.R - model.C)  # Daily profit

# Define the objective function
model.obj = pyo.Objective(expr=model.P, sense=pyo.maximize)

# Step 3: Define the constraints

# Constraint 1
model.c1 = pyo.Constraint(expr=model.x_eir == (model.x_ir + model.x_im) / model.x_of)

# Constraint 2
model.c2 = pyo.Constraint(expr=model.x_as == (model.k12 * model.x_aar) / (model.x_aar * model.x_adf + model.k13 * model.x_aar))

# Constraint 3
model.c3 = pyo.Constraint(expr=model.x_ay == model.x_of * (model.k1 + model.k2 * model.x_eir + model.k3 * model.x_eir**2))

# Constraint 4
model.c4 = pyo.Constraint(expr=model.x_mon == model.k4 + model.k5 * model.x_eir + model.k6 * model.x_eir**2 + model.k7 * (model.x_as - 89))

# Constraint 5
model.c5 = pyo.Constraint(expr=model.x_adf == model.k8 - model.k9 * model.x_f4pn)

# Constraint 6
model.c6 = pyo.Constraint(expr=model.x_f4pn == model.k10 + model.k11 * model.x_mon)

# Constraint 7
model.c7 = pyo.Constraint(expr=78 <= model.x_as <= 99)

# Constraint 8
model.c8 = pyo.ConstraintList(
    [model.x_of >= 100, model.x_of <= 1000,
     model.x_ir >= 100, model.x_ir <= 1000,
     model.x_aar >= 10, model.x_aar <= 100,
     model.x_im >= 10, model.x_im <= 100,
     model.x_as >= 78, model.x_as <= 99,
     model.x_mon >= 85, model.x_mon <= 95,
     model.x_eir >= 0,
     model.x_adf >= 0,
     model.x_f4pn >= 0]
)

# Constraint 9
model.c9 = pyo.Constraint(expr=model.x_ay == 1.22 * (model.x_of + model.x_im))

# Solve the model
solver = pyo.SolverFactory('glpk')
results = solver.solve(model)

# Print the results
print(f"Daily profit: {model.obj():.2f}")
print("Variable values:")
for v in model.component_objects(pyo.Var, active=True):
    print(f"  {v.name}: {getattr(model, v.name)()}")

PyomoException: Cannot convert non-constant Pyomo expression (78  <=  x_as) to bool.
This error is usually caused by using a Var, unit, or mutable Param in a
Boolean context such as an "if" statement, or when checking container
membership or equality. For example,
    >>> m.x = Var()
    >>> if m.x >= 1:
    ...     pass
and
    >>> m.y = Var()
    >>> if m.y in [m.x, m.y]:
    ...     pass
would both cause this exception.

### Solution Correctness